# Model Comparison for Dynamic Nelson Siegel Estimation

This notebook compares different estimation methods for the Dynamic Nelson Siegel model using the US Yield Curve data from 1972 to 2000.

In [1]:
import pandas as pd
from src.dns.models.cross_sectional import estimate_cross_sectional_parameters
from src.dns.data.loader import load_data

# Load the data
url = 'https://www.dropbox.com/s/inpnlugzkddp42q/bonds.csv?dl=1'
df = pd.read_csv(url, sep=';', index_col=0)

# Display the first few rows of the dataset
df.head()

## Approach 1: Cross-Sectional DNS Parameter Estimation

In this section, we will estimate the cross-sectional DNS parameters (βs) at each point in time and model their dynamics using a VAR.

In [2]:
# Estimate cross-sectional parameters
cross_sectional_results = estimate_cross_sectional_parameters(df)

# Display the results
cross_sectional_results

## Conclusion

This notebook has demonstrated the first approach for estimating the Dynamic Nelson Siegel model using cross-sectional parameters. Further comparisons with other methods will be conducted in subsequent sections.